# Riduzione della dimensionalità di dati RNA-Seq
I dati molecolari sono estratti attraverso una procedura di sequenziamento dell'RNA chiamata RNA-Seq. I file contenenti i dati molecolari iniziano tutti con la dicitura "data_RNA_Seq_v2_mRNA_" e contengono gli stessi dati normalizzati con procedure diverse. Questo perché i dati molecolari possono essere normalizzati in tanti modi e non si sa a priori quale tipologia di normalizzazione funziona meglio per la costruzione dell'autoencoder. Ognuno di questi file contiene una matrice in cui vi è una riga per ogni gene (feature di input) e una colonna per ogni paziente. Le prime due colonne contengono i nomi delle righe in formato diverso. La prima riga contiene i nomi delle colonne (tranne le prime due colonne i restanti nomi sono i codici dei campioni). I codici dei campioni sono sequenze di 15 caratteri. L'unico file che differisce leggermente è il file "data_RNA_Seq_v2_raw_counts.txt" che contiene i dati grezzi non normalizzati. In questo caso vale sempre che le righe sono i geni e le colonne sono i pazienti. Ma in questo specifico caso, solo la prima colonna contiene i nomi dei geni. Dalla seconda colonna in poi trovi i singoli campioni. Ogni campione è qui identificato da un codice lungo 28 caratteri, di cui i primi 15 corrispondono al codice del campione analogo a quello indicato negli altri file.

## Estrazione dei dati dal dataset RNA-Seq

Le strutture degli autoencoders sono disponibili nel modulo `autoencoder`. Le scelte architetturali ed implementative sono descritte all'interno della tesi. Prima di tutto, estraiamo i dati dal dataset RNA-Seq attraverso il modulo `ngs`. Utilizzeremo di default il file `data_RNA_Seq_v2_mRNA_median_all_sample_ref_normal_Zscores.txt`, che ha fornito i risultati di compressione migliori. Il metodo `ngs.get_ds()`  restituisce il dataset già preprocessato (attraverso gli step descritti nella tesi).

In [1]:
import pandas as pd 
import numpy  as np 

from modules import ngs  
from modules.autoencoder import *
from modules.preprocess  import *
from modules.utilities   import *
from modules.postprocess import *

ds = ngs.get_ds()
ds.head()

C:\Users\Charlemagne\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Entrez_Gene_Id,100133144,100134869,10357,10431,155060,26823,280660,340602,388795,390284,...,55055,11130,7789,158586,79364,440590,79699,7791,23140,26009
TCGA-3C-AAAU-01,1.0625,1.4054,-2.6318,-2.3685,3.9877,-2.1170,-4.3428,1.2983,-1.2612,-0.7965,...,1.9072,3.2236,1.4119,2.6413,1.4366,2.1646,-0.1948,-0.6725,0.2801,0.1443
TCGA-3C-AALI-01,0.3242,1.8883,-1.1750,-0.9363,2.1972,0.2431,-0.2931,14.2840,1.7157,-2.3744,...,4.8779,3.2788,-1.1521,-0.481,0.3201,1.9051,-2.0915,0.5403,-1.2083,-3.126
TCGA-3C-AALJ-01,0.6145,0.8671,2.8280,2.9686,2.4923,-2.1170,-4.3428,-1.9741,0.7886,-1.5576,...,1.0158,4.7992,-2.878,0.1908,-2.6322,2.4101,-2.4353,0.5178,-2.5848,-2.435
TCGA-3C-AALK-01,0.6655,1.1569,2.4738,0.9625,2.3449,-1.0068,-4.3428,-1.9741,0.4009,-0.6131,...,1.7551,2.7002,-1.4273,-0.5757,-2.0154,1.784,-2.0744,0.63,-3.3569,-1.4565
TCGA-4H-AAAK-01,-0.0534,1.5834,-0.2282,0.4232,0.7330,-0.9801,-4.3428,-0.8161,0.9240,-1.3989,...,1.836,2.7139,-1.8134,-0.6962,-0.984,-0.6448,-1.406,-0.2909,-3.0985,-0.7518


Estraiamo il numero di features, che servirà in futuro.

In [ ]:
nfeatures = ds.shape[1]

Salviamo il nome del file contente il dataset.

In [ ]:
dataset_file = ngs.get_data_sources()[1]

Salviamo il nome della cartella che conterrà i risultati.

In [ ]:
results_directory = 'results/ngs/blind'

## Estrazione degli autoencoders

Estraiamo gli autoencoders (blind):

In [ ]:
autoencoders_structures = blind_autoencoders(nfeatures)

## Preparazione dei dati di training

La tesi descrive accuratamente come avviene lo split e come avviene lo scaling (min-max normalization). 

In [ ]:
x_train, x_valid = train_test_split(ds, test_perc = .1)
x_train, x_test  = train_test_split(x_train, test_perc = .1)
x_train, x_test, scaler = normalize_sets(x_train, x_test)
x_valid = normalize_with_pretrained_scaler(scaler, x_valid)

## Embedding space di dimensione 150

Creiamo il modello `BLA150`: BLind Autoencoder 150 (embedding space dimension)

In [ ]:
emb150 = autoencoders_structures.get(150)
enc150 = emb150.get('encoder')
dec150 = emb150.get('decoder')
encpos = emb150.get('encoder_position')

BLA150 = generate_deep_autoencoder(nfeatures, enc150, dec150) 

In [ ]:
from keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor='val_loss', verbose=1, patience=20)

BLA150_training_hist = BLA150.fit(x_train, x_train,
            epochs=50, 
            batch_size=256,
            shuffle=True,
            verbose=1,
            callbacks=[earlystopping],
            validation_data=(x_test, x_test))

plot_history(BLA150_training_hist)

### Valutazione modello BLA150 

In [ ]:
training_loss = BLA150_training_hist.history['val_loss'][-1]
loss, mae, mse = BLA150.evaluate(x_valid, x_valid, verbose=0)
loss, mae, mse

In [ ]:
visualize_as_images(x_valid, BLA150, 10)

### Esportazione del dataset compresso da BLA150

In [ ]:
BLA150_encoder = extract_encoder(BLA150, encpos)
BLA150_ds = pd.DataFrame(BLA150_encoder.predict(np.asarray(ds).astype('float32')))
BLA150_ds.index = ds.index
BLA150_ds.head()

In [ ]:
save_compressed_dataset(BLA150_ds, loss, results_directory, dataset_file)

## Embedding space di dimensione 50

Creiamo il modello `BLA50`: BLind Autoencoder 50 (embedding space dimension)

In [ ]:
emb50 = autoencoders_structures.get(50)
enc50 = emb50.get('encoder')
dec50 = emb50.get('decoder')
encpos = emb50.get('encoder_position')

BLA50 = generate_deep_autoencoder(nfeatures, enc50, dec50) 

In [ ]:
from keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor='val_loss', verbose=1, patience=20)

BLA50_training_hist = BLA50.fit(x_train, x_train,
            epochs=50, 
            batch_size=256,
            shuffle=True,
            verbose=0,
            callbacks=[earlystopping],
            validation_data=(x_test, x_test))

plot_history(BLA50_training_hist)

### Valutazione modello BLA50 

In [ ]:
training_loss = BLA50_training_hist.history['val_loss'][-1]
loss, mae, mse = BLA50.evaluate(x_valid, x_valid, verbose=0)
loss, mae, mse

In [ ]:
visualize_as_images(x_valid, BLA50, 10)

### Esportazione del dataset compresso da BLA50

In [ ]:
BLA50_encoder = extract_encoder(BLA50, encpos)
BLA50_ds = pd.DataFrame(BLA50_encoder.predict(np.asarray(ds).astype('float32')))
BLA50_ds.index = ds.index
BLA50_ds.head()

In [ ]:
save_compressed_dataset(BLA150_ds, loss, results_directory, dataset_file)

## Embedding space di dimensione 25

Creiamo il modello `BLA25`: BLind Autoencoder 25 (embedding space dimension)

In [ ]:
emb25 = autoencoders_structures.get(25)
enc25 = emb25.get('encoder')
dec25 = emb25.get('decoder')
encpos = emb25.get('encoder_position')

BLA25 = generate_deep_autoencoder(nfeatures, enc25, dec25) 

In [ ]:
from keras.callbacks import EarlyStopping

earlystopping = EarlyStopping(monitor='val_loss', verbose=1, patience=20)

BLA25_training_hist = BLA25.fit(x_train, x_train,
            epochs=50, 
            batch_size=256,
            shuffle=True,
            verbose=1,
            callbacks=[earlystopping],
            validation_data=(x_test, x_test))

plot_history(BLA25_training_hist)

### Valutazione modello BLA25

In [ ]:
training_loss = BLA25_training_hist.history['val_loss'][-1]
loss, mae, mse = BLA25.evaluate(x_valid, x_valid, verbose=0)
loss, mae, mse

In [ ]:
visualize_as_images(x_valid, BLA25, 10)

### Esportazione del dataset compresso da BLA25

In [ ]:
BLA25_encoder = extract_encoder(BLA25, encpos)
BLA25_ds = pd.DataFrame(BLA25_encoder.predict(np.asarray(ds).astype('float32')))
BLA25_ds.index = ds.index
BLA25_ds.head()

In [ ]:
save_compressed_dataset(BLA150_ds, loss, results_directory, dataset_file)